# Temporal Data Playbook

**What this notebook shows**
- Interactive visualization with Altair
- Statistical testing with SciPy

**Data**
- Remote sources: https://data.open-power-system-data.org/time_series/2019-06-05, https://gist.githubusercontent.com/isha211/300f50a991019f32574b702e7c274fab/raw/629f396c87c264d2714a8dd8f248623c55766c0c/us_presidents.csv (+1 more)


## 🛒 1. Prepare 

Before we are able to do anything, we need to include the libraries that we are working with (as always):

In [ ]:
import pandas as pd
import altair as alt
import scipy.signal # for the LOESS fitting

In [ ]:
print(alt.__version__)

### Parse dates and times

In its most basic form, time series data contain a quantitative measure that changes over time. To reference a time point we use [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) of Pandas as the data type for temporal entities.

With **`to_datetime()`** you can create a Timestamp with a string containing a date and/or time. Pandas can infer the date and time from various date/time strings. Let's start with the present:


In [ ]:
pd.to_datetime("now")

We can pass a range of date formats and Pandas will guess which numbers refer to years, months, days, hours, etc.:

In [ ]:
pd.to_datetime('2020-05-07 3pm')

When expressing dates and times in written language, there is an ambiguity between the order of different entities. The most frequent ambiguity concerns the order of days and months, as they are typyically both expressed in double-digit numbers, unlike years that tend to be expressed with four digits. However, date conventions vary across the world.  For example, the following date might be interpreted differently depending on the country; it may refer to Saint Nicholas Day in 1929 or Anne Frank's birthday:

In [ ]:
pd.to_datetime('12.6.1929')

To clarify towards Pandas that the first number refers to the day, you can add the parameter **`dayfirst`**:

In [ ]:
pd.to_datetime('12.6.1929', dayfirst=True)

The method `to_datetime()` can also handle an array of date strings; it will return a `DatetimeIndex`, which is crucial for temporal indexing with Pandas.

In [ ]:
sessions=["2.4.2020", "9.4.2020", "16.4.2020", "23.4.2020", "7.5.2020", "14.5.2020", "28.5.2020", "4.6.2020", "11.6.2020", "25.6.2020", "2.7.2020", "9.7.2020"]
pd.to_datetime(sessions, dayfirst=True)

If you want to make extra sure that the date/time string is parsed correctly and quickly, you can pass a fixed **`format`** for the date/time strings to be parsed:

In [ ]:
pd.to_datetime('2020-05-07', format="%Y-%m-%d")

*see [list of format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)!*

### Load time series data

In this playbook we will be analyzing energy use in Germany from 2015 until 2018. 

First we will load the freely accessible data from the [OPSD project](https://open-power-system-data.org). `read_csv()` has a convenient feature, which lets you specify the column containing date/time information.

In [ ]:
# data downloaded from OPSD - see filter elements on this page:
# https://data.open-power-system-data.org/time_series/2019-06-05

df = pd.read_csv('https://gist.githubusercontent.com/isha211/4f5c03158c3c821ee2e428e84165cb0c/raw/ea34c6fb23aded0cd4659c364435e594305173ce/german_energy_data.csv', parse_dates=['datetime']) # parse timestamp column

✏️ *Check out the structure and content of `opsd` — Hint: `info()` and `head()` are your friends, and `tail()` which gives the end of the list*

In [ ]:
df.head()


The time parsing resulted in one hour to be included from 2014. This is because this dataset covers the years 2015-2018 according to German time, while the generic UTC time is one hour 'behind'. To avoid having this stray hour in the old year, we revert the timestamps back to Germany's timezone:

In [ ]:
df["datetime"] = df["datetime"].dt.tz_convert("Europe/Berlin")
df.head()

Did you notice that solar energy appears to be zero at the beginning and at the end of the DataFrame? Take a look at the time and you'll know why.

In order to get a random data `sample()` there is a method of the same name, which gives us a random set of rows:

In [ ]:
df.sample(10)

Now, let's take a look at the data types and values contained in our DataFrame:

In [ ]:
df.info()

The values in the columns `load`, `solar` and `wind` are provided in the unit megawatt (MW) as integers.

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame:

In [ ]:
df = df.set_index("datetime")

When we run `df.info()` again, you will see that the DataFrame now has a `DatetimeIndex`:

In [ ]:
df.info()

The `DatetimeIndex` provides a few handy methods to extract temporal units such as months, days, week of the year, etc.: 

In [ ]:
df.index.year.unique()

*Look here for syntax to extract any other [temporal attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)*

## 🕒 2. Process

The dataset contains over 35000 rows each of which contains data on overall energy load and renewable energy production. To make sense of all this data, we need to process the time series data into various chunks and sizes!


### Query time points and spans

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

One useful function of a temporal index, is its querying function. We can quickly extract the rows for a given time point or period.

Let's get the data for the day with the most daylight of 2017, i.e., the summer solstice or midsummer:

In [ ]:
df.loc["2017-06-21"]

Above query is an example of partial-string indexing: while our `DateTime` column actually contains time information as well, you can query it quickly (!) with just the date, or even a shorter query:

In [ ]:
df.loc["2017-06"]

You can also query a time period. Do you remember the storm Xavier that hit Germany in early October 2017? Let's retrieve the data around this time:

In [ ]:
df.loc["2017-10-04":"2017-10-06"]

### Aggregate values along time

The data we retrieved from OPSD, comes in the granularity of hours. To better understand the data it can be useful to reduce the data resolution and consider, for example, the total energy used/produced everyday or the daily averages over the course of entire years.

The DataFrame's `resample()` [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) method provides a concise and quick way of aggregating temporally indexed data along time units. Here we create a DataFrame with summed up values for each year aggregated from the original dataset:

In [ ]:
sums = df.resample("Y").sum()
sums

The resample operations can also be carried out one after another. For example, we might want to know how weekly energy use/production varies between the quarters:

In [ ]:
# first we create the sums per week
weekly_sums = df.resample("W").sum()
# then we generate the weekly means for each quarter
quarterly_means = weekly_sums.resample("Q").mean()
# for readability we'll revert the values back to integers
quarterly_means.astype(int)

## 🥗 3. Present

Enough data processing. It's time for visualization!

### Time spans

One of the first time visualizations was [*A Chart of Biography*](https://en.wikipedia.org/wiki/A_Chart_of_Biography) (1765) by Joseph Priestley. Let's create a similar visualization of the US presidencies since World War II. First we load the CSV file with `pd.read_csv()`:

In [ ]:
presidents = pd.read_csv("https://gist.githubusercontent.com/isha211/300f50a991019f32574b702e7c274fab/raw/629f396c87c264d2714a8dd8f248623c55766c0c/us_presidents.csv")

The start and end times of the presidencies are given as four-digit integers, i.e., years. To convey towards Pandas and Altair that the start and end columns are actually dates, we need to parse them using the `to_datetime()` method:

In [ ]:
presidents['start'] = pd.to_datetime(presidents['start'], format="%Y") 
presidents['end'] = pd.to_datetime(presidents['end'], format="%Y") 

The following chart consists of two parts: `bars` and `labels`. The former will be the main bar chart representing the time spans of the presidencies, and the latter will add the presidents' names. This way we can position the labels right next to the bars, much nicer!

In [ ]:
# we add height as a parameter to make the time spans appear less clunky
bars = alt.Chart(presidents).mark_bar(height=5).encode(
    # this time we need two columns for x-position
    x='start', x2='end',
    # we sort by start dates and hide the axis, as names are added below
    y=alt.Y('name', sort='x', axis=None),
    color="party"
)

# the labels are added to the barchart using the mark_text command
labels = bars.mark_text(align='right', dx=-5).encode(text='name')

# both charts integrated into a layered chart through the magic of a plus sign
(bars + labels).properties(width=800, height=400)

### Overall trends

Next, we are going to return to the energy time series data that we prepared above. Remember that the original dataset has an hourly resolution resulting in too many data points to visualize at once. Altair itself handles at most 5000 data rows.

To reduce the dataset into a manageable size, we  will create daily sums with the `resample()` method of the DateTimeIndex:

In [ ]:
days = df.resample("D").sum()

To visualize the contents of the DataFrame with Altair, we turn the date/time information in the index into its own column `datetime` again and transform the DataFrame from [wide to long form](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data) using the `melt()` method. We pass the name of the `datetime` column to the melt method so that it is treated as an index variable. To keep it short, these two steps `reset_index()` and `melt("datetime")` are done in one go right when we create the `Chart()`, which is what we are going to the in the rest of the playbook.

Now let's start with a scatterplot visualization of the daily data:

In [ ]:
# Break down the long form of the data so we can see that it works
days.reset_index().melt("datetime",var_name='Type')

In [ ]:
alt.Chart(days.reset_index().melt("datetime")).mark_circle().encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

With this our eyes can already see several patterns going on. Some are more dictinct than others. Interestingly, the load data (blue) seems to have two separate, but parallel curves…

✏️ *There is a lot of overplotting going on. Reduce the `size` and `opacity` of all dots, by passing these as parameters to `mark_circle`!*

Next, we are going to connect the dots and create a line chart form this data. So basically the same code as above, except we're now using `mark_line()` instead of `mark_circle`:

In [ ]:
alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

This chart already shows a lot: we can see weekly patterns—the jittery up and down—in the load (blue) and the seasonal patterns in the form of broad waves in the load and solar curves (blue and yellow). Right around the turn of the year we see a drop of energy load.

While above line chart is truthful to the local fluctuations, it makes it hard to actually grasp the up and down over the course of months and years. Let's change the sampling from days to months to examine the overall patterns in the data.

In [ ]:
months = df.resample("M").sum()

# to make the line a bit smoother, we include an interpolate parameter
alt.Chart(months.reset_index().melt("datetime")).mark_line(opacity=0.75, interpolate="basis").encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

What do you think? The fine-grained jitter is now gone and we might have lost too much detail. In fact, first downsampling the data and then including an interpolation is maybe giving it too much of a treatment (like overusing Photoshop's blur function). For example, the load minima around the turns of the year are not visible as the overall energy load in the winter months appears larger. One the other hand, the solar curve has become an almost perfect sine wave, probably relating to the position of the sun across the year.

💡 *Play with different granularities in he `resample()` step. For example, use **W** for week or **Q** for quarter!*

One way to integrate the local and global patterns is to create a layered graph, as we have already done with the presidents' names above. This time we are combining a line chart of the days with a line chart of monthly averages.

Note how we ensure that both DataFrames fit the same data scale. As we're looking at energy values per day, we first generate the daily sums and on this basis the monthly means:

In [ ]:
days = df.resample("D").sum()
months = days.resample("M").mean()

Next we create the line charts and combine the two again with the **+** operator:

In [ ]:
chart1 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.25).encode(
    x='datetime',
    y='value',
    color='variable',
).properties(width=800, height=400)

chart2 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=1).encode(
    x='datetime',
    y='value',
    color='variable',
)

chart1 + chart2

With this view we already get a good sense of the overall time patterns, while still seeing some of the particular variations.

💡 *You can add the `.interactive()` directive to one of these charts to make them zoomable!*

### Rolling windows

While the `resample()` method takes a broad brush and results in a reduced dataset and a chart with smooth curves, `rolling()` offers an alternative way of smoothing out local outliers without actually reducing the resolution of the dataset.

The first parameter determines the window size, by positioning the window at the `center` values are considered in both directions of the current date/time, and `win_type` determines how the values across the window are weighted; with the `triang` option the values further away contribute less:

In [ ]:
rolling = days.rolling(60, center=True, win_type="triang").mean()

chart1 = alt.Chart(rolling.reset_index().melt("datetime")).mark_line(strokeWidth=1.5, opacity=1).encode(
    x='datetime', y='value', color='variable',
).properties(width=800, height=400)

# same as the two charts in previous code cell, except more transparent
chart2 = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1, opacity=0.1).encode(
    x='datetime', y='value', color='variable',
)

chart3 = alt.Chart(months.reset_index().melt("datetime")).mark_line(interpolate="basis", opacity=.25).encode(
    x='datetime', y='value', color='variable',
)

chart1 + chart2 + chart3

Above you see the two lines for daily sums and monthly averages from the previous cell (slightly more transparent), on top of which you can see the time curve generated with a rolling window. It is quite apparent that this curve still features more pronounced dips around the end-of-year periods and elsewhere.


💡 *Play around with different window sizes and other parameters in the first line in above cell!*

### Detailed views

Some trends seem to pan out at much lower scales, which require higher levels of detail. For example, at the beginning of 2018 we see a peak in the wind energy production. In the winter of 2017/2018 Germany and most of Europe actually witnessed several storms. And so it also happened in the first days of 2018, when [Burglind (a.k.a. Eleanor)](https://en.wikipedia.org/wiki/Storm_Eleanor_(2018)) passed by. Let's take a closer look at the first week of January 2018. For this we extract a subset of the DataFrame using our tried and tested `loc[]`, which lets us select specific time spans:

In [ ]:
burglind = df.loc["2018-01-01":"2018-01-07"]

alt.Chart(burglind.reset_index().melt("datetime")).mark_line().encode(
    x='datetime',
    y='value',
    color='variable'
).properties(width=800, height=400)

### High-level bars

In contrast to the detailed close-up, we might also want to look at much more general patterns. Let's say we are curious about the total production of renewable energy over the four years in the dataset. 

For this we `resample` the original data from hourly data resolution into summed up values per year:

In [ ]:
years = df.resample("Y").sum()

Next we create a new column, which contains the summed up values of solar and wind production.

In [ ]:
years["renewable"] = years["solar"]+years["wind"]

We `drop` the other value columns:

In [ ]:
renewable = years.drop(columns=["load", "solar", "wind"])

Extract the years and create a specific column:

In [ ]:
renewable["year"] = years.index.year
renewable

And we remove the datetime index, and this time discard it (i.e., `drop=True`), as we have the `year` column already:

In [ ]:
renewable = renewable.reset_index(drop=True)
renewable

Now we can create a simple barchart from this:

In [ ]:
alt.Chart(renewable).mark_bar(width=30, fill="purple").encode(
    x='year:O',
    y='renewable:Q'
).properties(width=300, height=300)

### Cyclical patterns

Previously, we did notice a few recurring patterns in the energy load. One pattern might be due to the weekly work-and-leasure patterns. 

So let's take a closer look at the average energy `load` over the course of a week:


In [ ]:
# remove solar and wind columns
weekdays = df.drop(columns=["solar", "wind"])

# add a column for weekdays
weekdays["weekday"] = weekdays.index.weekday

This time we do not use `resample()` to change the data granularity, but `groupby()` to study recurring data patterns, because we are actually interested in the energy load for each *generic* weekday across the entire dataset:

In [ ]:
weekdays = weekdays.groupby("weekday").mean()

alt.Chart(weekdays.reset_index()).mark_bar(width=20).encode(
    x='weekday:O',
    y='load:Q'
).properties(width=300, height=300)

It's quite apparent that energy load is lower during weekends.

### Interactive selection

If you want it all: A high-level overview and detailed close-ups, you might need two coordinated views:

In [ ]:
weeks = df.resample("W").sum()
days = df.resample("D").sum()

brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(weeks.reset_index().melt("datetime")).mark_area(interpolate="basis").encode(
    x = alt.X('datetime:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=800, height=50).add_selection(brush)

lower = alt.Chart(days.reset_index().melt("datetime")).mark_line(strokeWidth=1).encode(
    x = alt.X('datetime:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=800, height=300)

upper & lower

Note: The small stacked graph above the main chart is interactive, you can adjust the viewport of the line graph by dragging a time span with it.

## Your turn

Apply your understanding of the above topics and answer the following 6 questions

In [ ]:
# 1 - Look at the documentation to translate the string to datetime using the correct format for VE day (replace the question marks below)

# pd.to_datetime('8.5.1945 23:01', format="???")
pd.to_datetime('8.5.1945 23:01', format="%d.%m.%Y %H:%M")

In [ ]:
# 2 - Append code to `df.index` below that shows what time zone the data is in (replace the question marks below)

# df.index.??
df.index.tz

In [ ]:
# 3 - Use date slicing/indexing (.loc) to find the mean of solar energy from 
# 21 June 2018 and the mean of the solar energy from 20 December 2018

summer_mean = df.loc["2018-06-21", "solar"].mean()
print(f"Mean solar energy on 21 June 2018: {summer_mean:.2f} MW")

winter_mean = df.loc["2018-12-20", "solar"].mean()
print(f"Mean solar energy on 20 December 2018: {winter_mean:.2f} MW")

In [ ]:
# 4 - How did daily sum energy use/production change over the years? Calculate the daily sum and then the yearly average. Then show the resulting dataframe of the yearly averages.

daily_sums = df.resample("D").sum()

yearly_averages = daily_sums.resample("Y").mean()

yearly_averages

In [ ]:
# 5 - Create a horizontal bar chart showing total renewable energy over the years 


years = df.resample("Y").sum()
years["renewable"] = years["solar"] + years["wind"]

renewable_data = years[["renewable"]].copy()
renewable_data["year"] = years.index.year
renewable_data = renewable_data.reset_index(drop=True)


alt.Chart(renewable_data).mark_bar(height=30, color="green").encode(
    x='renewable:Q',
    y=alt.Y('year:O', axis=alt.Axis(title='Year'))
).properties(width=600, height=250, title="Total Renewable Energy by Year")

In [ ]:
# 6 - Show the wind energy for the month of Dec 2018 along with the Savitzky–Golay filter line overlayed on top as a line chart. Set your window for one week of data. Play around with the polynomial order until you 
# get something you like. 

dec_2018 = df.loc["2018-12", "wind"].copy()

wind_data = pd.DataFrame({
    'datetime': dec_2018.index,
    'wind': dec_2018.values
})

window_size = 168
poly_order = 3
wind_data['smoothed'] = scipy.signal.savgol_filter(wind_data['wind'], window_size, poly_order)
wind_long = wind_data.melt('datetime', var_name='type', value_name='wind_energy')

alt.Chart(wind_long).mark_line().encode(
    x=alt.X('datetime:T', title='Date'),
    y=alt.Y('wind_energy:Q', title='Wind Energy (MW)'),
    color=alt.Color('type:N', 
                    scale=alt.Scale(domain=['wind', 'smoothed'], 
                                   range=['lightblue', 'darkblue']),
                    legend=alt.Legend(title='Type'))
).properties(
    width=800, 
    height=400,
    title='Wind Energy in December 2018 with Savitzky-Golay Filter'
)

## Sources

Tutorials & Examples
- [​​​​Tutorial: Time Series Analysis with Pandas by Jennifer Walker](https://www.dataquest.io/blog/playbook-time-series-analysis-with-pandas/)
- [Altair Interval Selection Example](https://altair-viz.github.io/gallery/interval_selection.html)

Documentation: Pandas
- [Time series / date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
- [Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html)
- [DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)
- [Time-aware rolling vs. resampling](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#time-aware-rolling-vs-resampling)

